In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import time

# === Configuration ===
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages"
os.makedirs(DATA_DIR, exist_ok=True)

label = input("Good").strip().lower()
SAVE_PATH = os.path.join(DATA_DIR, f"{label}.csv")

# === MediaPipe setup ===
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2, 
    min_detection_confidence=0.7
)

cap = cv2.VideoCapture(0)
data = []
frame_count = 0
saved_count = 0

print("📷 Starting capture in 3 seconds...")
time.sleep(3)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    frame_count += 1

    # Detect hands
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # === If no hand detected ===
    if not results.multi_hand_landmarks:
        cv2.putText(frame, "No hands detected!", (10, 35),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Two-Hand Capture", frame)

        # Skip saving this frame (no data collected)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    # === If hand(s) detected, record landmarks ===
    row = []
    for hand_landmarks in results.multi_hand_landmarks:
        for lm in hand_landmarks.landmark:
            row.extend([lm.x, lm.y, lm.z])

    # Pad second hand if only one hand detected
    if len(results.multi_hand_landmarks) == 1:
        row.extend([0] * (21 * 3))

    row.append(label)
    data.append(row)
    saved_count += 1

    # Draw hands on frame
    for hand_landmarks in results.multi_hand_landmarks:
        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.putText(frame, f"Collecting: {label}", (10, 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Samples: {saved_count}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.imshow("Two-Hand Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# === Save dataset ===
columns = []
for hand in ["L1_", "L2_"]:
    for i in range(21):
        columns += [f"{hand}x{i}", f"{hand}y{i}", f"{hand}z{i}"]
columns.append("label")

df = pd.DataFrame(data, columns=columns)

if not df.empty:
    df.to_csv(SAVE_PATH, index=False)
    print(f"\n✅ Dataset saved to {SAVE_PATH}")
    print(f"🧮 Frames processed: {frame_count}")
    print(f"💾 Valid samples collected: {saved_count}")
else:
    print("🚫 No valid samples collected (no hands detected). File not saved.")

📷 Starting capture in 3 seconds...


c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



✅ Dataset saved to C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\BasicPhrasesSignLanguages\good.csv
🧮 Frames processed: 369
💾 Valid samples collected: 261


In [36]:
import pandas as pd
import glob
import os

# Folder containing your individual datasets
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages"

# Find all CSV files (each phrase dataset)
all_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
print(f"📁 Found {len(all_files)} dataset files")

df_list = []

for file in all_files:
    try:
        df = pd.read_csv(file)
        if df.empty:
            print(f"⚠️ Skipped empty file: {os.path.basename(file)}")
            continue
        df_list.append(df)
        print(f"✅ Loaded {os.path.basename(file)} ({df.shape[0]} samples)")
    except Exception as e:
        print(f"❌ Error reading {os.path.basename(file)}: {e}")

# Merge them all together
if df_list:
    final_df = pd.concat(df_list, ignore_index=True)
    combined_path = os.path.join(DATA_DIR, "combined_basic_phrases_dataset.csv")
    final_df.to_csv(combined_path, index=False)

    print("\n✅ Combined dataset created successfully!")
    print(f"📄 Saved to: {combined_path}")
    print("🧮 Total samples:", final_df.shape[0])
    print("🏷️ Labels:", final_df['label'].unique())
else:
    print("🚫 No valid datasets found to merge.")


📁 Found 6 dataset files
✅ Loaded combined_basic_phrases_dataset.csv (3624 samples)
✅ Loaded good.csv (261 samples)
✅ Loaded hello.csv (173 samples)
✅ Loaded hey.csv (244 samples)
✅ Loaded iloveyou.csv (214 samples)
✅ Loaded thankyou.csv (230 samples)

✅ Combined dataset created successfully!
📄 Saved to: C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\combined_basic_phrases_dataset.csv
🧮 Total samples: 4746
🏷️ Labels: [nan 'thankyou' 'hello' 'iloveyou' 'good' 'Hey']


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import pickle

DATA_PATH = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\combined_basic_phrases_dataset.csv"
df = pd.read_csv(DATA_PATH)

# 🧹 Step 1: Clean Data
print("Checking for NaN values...")

nan_counts = df.isna().sum().sum()
if nan_counts > 0:
    print(f"⚠️ Found {nan_counts} missing values — cleaning dataset...")
    df = df.dropna()
else:
    print("✅ No missing values detected.")

# Ensure numeric values only
X = df.drop('label', axis=1)
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

y = df['label']

# 🧪 Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 🧠 Step 3: Train Model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 📊 Step 4: Evaluate
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 💾 Step 5: Save Model
with open("basic_phrases_twohand_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("💾 Model saved as 'basic_phrases_twohand_model.pkl'")


Checking for NaN values...
⚠️ Found 598283 missing values — cleaning dataset...


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [35]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

# Load trained model
with open("basic_phrases_twohand_model.pkl", "rb") as f:
    model = pickle.load(f)

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        row = []
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                row.extend([lm.x, lm.y, lm.z])
        if len(results.multi_hand_landmarks) == 1:
            row.extend([0] * (21 * 3))
        x = np.array(row).reshape(1, -1)
        prediction = model.predict(x)
        cv2.putText(frame, f"{prediction[0]}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)

        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Prediction - Two Hands", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Ple